<a href="https://colab.research.google.com/github/abhilb/Open-eye-closed-eye-classification/blob/main/Part2/MRL_Eye_Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, Input, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

In [82]:
tf.__version__

'2.7.0'

In [83]:
input_img = Input(shape=(32,32,1))
x = Conv2D(16, (3, 3), activation='relu', padding='same', name='conv1')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same', name='conv2')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same', name='conv3')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same', name='conv4')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same', name='conv5')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same', name='conv6')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same', name="decoder")(x)

autoencoder = Model(input_img, decoded, name="eye_autoencoder")
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [84]:
autoencoder.summary()

Model: "eye_autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 32, 32, 1)]       0         
                                                                 
 conv1 (Conv2D)              (None, 32, 32, 16)        160       
                                                                 
 max_pooling2d_33 (MaxPoolin  (None, 16, 16, 16)       0         
 g2D)                                                            
                                                                 
 conv2 (Conv2D)              (None, 16, 16, 8)         1160      
                                                                 
 max_pooling2d_34 (MaxPoolin  (None, 8, 8, 8)          0         
 g2D)                                                            
                                                                 
 conv3 (Conv2D)              (None, 8, 8, 8)       

In [85]:
!gdown https://drive.google.com/uc?id=1JkdFa4fj0DMrDHju7QqC4EiUfJD9Cjws

Downloading...
From: https://drive.google.com/uc?id=1JkdFa4fj0DMrDHju7QqC4EiUfJD9Cjws
To: /content/mrl_dataset.npz
100% 696M/696M [00:05<00:00, 126MB/s] 


In [86]:
dataset_path = Path('mrl_dataset.npz').absolute().resolve()
data = np.load(str(dataset_path))
X = data['data']
y = data['labels']
data.close()

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
print(f"Number of samples in training dataset: {X_train.shape[0]}")
print(f"Number of samples in testing dataset : {X_test.shape[0]}")

Number of samples in training dataset: 56881
Number of samples in testing dataset : 28017


In [88]:
opt = Adam(learning_rate=0.0001)
loss = BinaryCrossentropy()
metrics = ['accuracy']
callback = EarlyStopping(monitor='loss', patience=3)
autoencoder.compile(optimizer=opt, loss=loss, metrics=metrics)

In [89]:
from tensorflow.keras.utils import Sequence
from typing import Tuple

class DataGenerator(Sequence):
  def __init__(self, batch_size: int,
               dims: Tuple[int, int],
               data, labels):
    self.batch_size = batch_size
    self.dims = dims
    self.data = np.hstack((data, labels))
    self.nb_samples = self.data.shape[0]
    self.indexes = np.arange(self.nb_samples)
    self.on_epoch_end()

  def __len__(self):
    return self.nb_samples // self.batch_size

  def __getitem__(self, index):
    start_idx = index * self.batch_size
    end_idx = (index + 1) * self.batch_size
    data_slice = self.data[start_idx:end_idx, :]
    X = data_slice[:,0:-1].reshape(-1,*self.dims)
    y = data_slice[:,-1]
    return X, y

  def on_epoch_end(self):
    np.random.shuffle(self.data)


In [90]:
train_generator = DataGenerator(64, (32, 32, 1), X_train[:40000,:], y_train[:40000])
valid_generator = DataGenerator(32, (32, 32, 1), X_train[40000:,:], y_train[40000:])

In [91]:
history = autoencoder.fit(train_generator,
                          validation_data=valid_generator,
                          epochs=100,
                          shuffle=True)

Epoch 1/100


ValueError: ignored